In [ ]:
!wget https://github.com/youone04/TA-Music/raw/main/model6_vgg16.h5

--2023-05-30 23:52:47--  https://github.com/youone04/TA-Music/raw/main/model6_vgg16.h5
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/youone04/TA-Music/main/model6_vgg16.h5 [following]
--2023-05-30 23:52:48--  https://raw.githubusercontent.com/youone04/TA-Music/main/model6_vgg16.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75537640 (72M) [application/octet-stream]
Saving to: ‘model6_vgg16.h5’

model6_vgg16.h5     100%[===================>]  72.04M   218MB/s    in 0.3s    

2023-05-30 23:52:49 (218 MB/s) - ‘model6_vgg16.h5’ saved [75537640/75537640]



In [ ]:
!pip install pydub
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os 
from google.colab import files
from pydub import AudioSegment

# konversi WAV to Spectogram
import numpy as np
import librosa.display, os
import keras.utils as image
import matplotlib.pyplot as plt
from keras.applications.vgg16 import preprocess_input
from keras.applications import VGG16
from keras import models    

import matplotlib
matplotlib.use('agg')
from matplotlib import cm
import pylab

In [ ]:
# Remove all mp3 files in /content
def remove_all_files():
  for filename in os.listdir():
    if len(filename.split(".")) >= 2 and filename.split(".")[-1] != 'h5':
      try: 
        os.remove(filename)
      except:
        pass

# Upload file to /content
def upload_file():
  uploaded = files.upload()
  filename = next(iter(uploaded))
  return "".join(filename.split('.')[:-1])

# Convert mp3 to wav
def mp3_to_wav(filename_without_ext):
  """file_path: Location of the file"""
  mp3 = AudioSegment.from_mp3(f"{filename_without_ext}.mp3")
  mp3.export(f"{filename_without_ext}.wav", format="wav")


def create_spectrogram(audio_file, image_file):
    y, sr = librosa.load(audio_file, sr = 22050)

    # Pre-emphasis filter
    pre_emphasis = 0.97
    y = np.append(y[0], y[1:] - pre_emphasis * y[:-1])

    ms = librosa.feature.melspectrogram(y=y, sr=sr, 
                                        fmax = sr/2, # Maximum frequency to be used on the on the MEL scale
                                        n_fft=2048, 
                                        hop_length=512, 
                                        n_mels = 96, # As per the Google Large-scale audio CNN paper
                                        power = 2)
    log_ms = librosa.power_to_db(ms, ref=np.max)

    # menentukan ukuran gambar
    # pylab.figure(figsize=(3,3)) #asli
    pylab.figure(figsize=(2.16,2.16)) #modif
    pylab.axis('off') 
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    # librosa.display.specshow(log_power, cmap=cm.jet)
    librosa.display.specshow(log_ms, sr=sr)
    # pylab.savefig(image_file, dpi=56) #modi
    pylab.savefig(image_file) #asli
    pylab.close()
    
def wav_to_png(filename_without_ext):
    create_spectrogram(f"{filename_without_ext}.wav", f"{filename_without_ext}.png")

def mp3_to_spectogram():
  remove_all_files()
  filename_without_ext = upload_file()
  mp3_to_wav(filename_without_ext)
  wav_to_png(filename_without_ext)
  return filename_without_ext

def predict_music():
  filename_without_ext = mp3_to_spectogram()
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(216, 216, 3))
  model = models.load_model(filepath="/content/model6_vgg16.h5")
  class_labels = ['dewasa', 'anak']
  x = image.load_img(f"{filename_without_ext}.png", target_size=(216, 216))
  plt.xticks([])
  plt.yticks([])
  plt.imshow(x)

  x = image.img_to_array(x)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  y = base_model.predict(x)
  predictions = model.predict(y)

  for i, label in enumerate(class_labels):
      print(f'{label}: {predictions[0][i]}')

In [ ]:
predict_music()

Saving SEPANJANG JALAN KENANGAN (Is Haryanto  Tetty Kadi) - T'KOES BAND.mp3 to SEPANJANG JALAN KENANGAN (Is Haryanto  Tetty Kadi) - T'KOES BAND.mp3


1/1 [==============================] - 1s 617ms/step


1/1 [==============================] - 0s 60ms/step
dewasa: 0.886347234249115
anak: 0.11365284025669098
